In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
import schedule
import time
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import os
from multiprocessing import Pool, cpu_count
import datetime

In [9]:
# Code OK 
# Data type carte img gaz
# URL de la page web
url_image = "http://www.prevair.org/donneesmisadispo/public/?C=N;O=A"
# Path driver
folder_image= "C:/Users/sofia/Desktop/4iabd/pa/data/image"

# Chemins de destination des images téléchargées
maxj_folder = folder_image +"/maxj/"
moyj_folder = folder_image +"/moyj/"

# Liste des polluants
polluants = ["NO2", "O3", "PM10", "PM25"]

# Créer les sous-dossiers pour chaque polluant
for folder_image in [maxj_folder, moyj_folder]:
    for polluant in polluants:
        os.makedirs(os.path.join(folder_image, polluant), exist_ok=True)

# Récupérer le contenu de la page web
response = requests.get(url_image)

# Analyser le contenu de la page web avec BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Rechercher tous les liens de fichiers dans la page web
file_links = [link.get("href") for link in soup.find_all("a") if link.get("href").endswith(".jpg")]

# Télécharger les fichiers
for file_link in file_links:
    # Construire l'URL de téléchargement
    download_url = "http://www.prevair.org/donneesmisadispo/public/" + file_link

    # Construire le chemin de destination du fichier
    filename = os.path.basename(download_url)
    if "MAXJ" in filename:
        dest_folder = maxj_folder
    elif "MOYJ" in filename:
        dest_folder = moyj_folder
    else:
        continue # ignorer les fichiers qui ne contiennent ni MAXJ ni MOYJ

    # Trouver le polluant dans le nom de fichier
    for polluant in polluants:
        if polluant in filename:
            polluant_folder = os.path.join(dest_folder, polluant)
            break
    else:
        continue # ignorer les fichiers qui ne contiennent aucun des polluants

    dest_path = os.path.join(polluant_folder, filename)

    # Vérifier si le fichier existe déjà
    if os.path.isfile(dest_path):
        print(f"Le fichier {filename} a déjà été téléchargé.")
        continue
    # Télécharger le fichier
    response = requests.get(download_url)
    with open(dest_path, "wb") as f:
        f.write(response.content)
        print(f"Le fichier {filename} a été téléchargé avec succès.")

Le fichier PREVAIR.analyse.20230601.MAXJ.NO2.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MAXJ.O3.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MAXJ.PM10.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MAXJ.PM25.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MOYJ.NO2.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MOYJ.O3.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MOYJ.PM10.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230601.MOYJ.PM25.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230602.MAXJ.NO2.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230602.MAXJ.O3.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230602.MAXJ.PM10.public.jpg a été téléchargé avec succès.
Le fichier PREVAIR.analyse.20230602.MAXJ.PM25.public

In [15]:
# OK 
# Récupération des données Pollens par commune
url_pollens = "https://api.airparif.fr/pollens/bulletin"

# Récupération de la date du jour
date_du_jour = datetime.date.today().strftime('%d-%m-%Y')

headers = {
    "X-Api-Key": "06a92dff-35b2-2308-cf68-95f2c55fc427"
}
res = requests.get(url_pollens, headers=headers)
dataAirParif = res.json()
print(dataAirParif)

# Extraction des données de la clé 'data'
data = dataAirParif['data']

# Initialisation d'une liste pour stocker les dictionnaires des colonnes
columns = []

# Extraction des noms de tous les pollens de la première observation
pollen_names = data[0]['taxons']

# Extraction des données de chaque observation
for observation in data:
    values = observation['valeurs']
    for departement, pollen_counts in values.items():
        row = {'departement': departement}
        for i, count in enumerate(pollen_counts):
            pollen_name = pollen_names[i]
            row[pollen_name] = count
        columns.append(row)

# Création du DataFrame

df_pollens = pd.DataFrame(columns)
df_pollens['departement'] = df_pollens['departement'].astype(int)
df_pollens['time'] = pd.Timestamp.today().date()

# Création du nom du fichier
nom_fichier = f"csv_pollen_{date_du_jour}.csv"
chemin_fichier = r"C:\Users\sofia\Desktop\4iabd\pa\data\pollens"+ nom_fichier

# Vérification si le fichier CSV existe déjà
if os.path.exists(chemin_fichier):
    print(f"Le fichier {nom_fichier} existe déjà.")
else:
    # Enregistrement du fichier CSV
    df_pollens.to_csv(chemin_fichier, index=False)
    print(f"Le DataFrame a été enregistré sous le nom de fichier : {chemin_fichier}")
    print(df_pollens)


{'data': [{'taxons': ['cypres', 'noisetier', 'aulne', 'peuplier', 'saule', 'frene', 'charme', 'bouleau', 'platane', 'chene', 'olivier', 'tilleul', 'chataignier', 'rumex', 'graminees', 'plantain', 'urticacees', 'armoises', 'ambroisies', 'pollens'], 'valeurs': {'75': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 2], '77': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 1, 0, 0, 3], '78': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 1, 0, 0, 2], '91': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 0, 0, 3], '92': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 0, 0, 3], '93': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 1, 0, 0, 3], '94': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 1, 0, 0, 3], '95': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 1, 0, 0, 3]}, 'commentaire': 'Dans le ciel des allergiques, on retrouvera des pollens de graminées avec un risque d’allergie de niveau moyen à élevé selon les départements. Attention aux belles journées ensole

In [17]:
#Ok
# Récupération des données des gazs polluents
download_folder = r'C:\Users\sofia\Desktop\4iabd\pa\data\gazs'

# Get a list of filenames of the files in the download folder
downloaded_files = os.listdir(download_folder)

url = "https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
for year_folder in soup.select('a[href$="/"]'):
    year_url = url + year_folder['href']
    response_year = requests.get(year_url)
    soup_year = BeautifulSoup(response_year.text, 'html.parser')
    for csv_file in soup_year.select('a[href$=".csv"]'):
        csv_url = year_url + csv_file['href']
        csv_filename = os.path.join(download_folder, csv_file['href'])
        if csv_file['href'] not in downloaded_files: # Check if the file is already downloaded
            with open(csv_filename, 'wb') as f:
                f.write(requests.get(csv_url).content)
                print(f"Le fichier {csv_file['href']} a été téléchargé avec succès.")
            downloaded_files.append(csv_file['href']) # Add the filename to the list of downloaded files

Le fichier FR_E2_2021-01-01.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-02.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-03.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-04.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-05.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-06.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-07.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-08.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-09.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-10.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-11.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-12.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-13.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-14.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-15.csv a été téléchargé avec succès.
Le fichier FR_E2_2021-01-16.csv a été téléchargé avec succès.
Le fichi

In [19]:
# Fonction pour lire un fichier CSV et renvoyer un DataFrame
def lire_csv(f):
    df = pd.read_csv(os.path.join(chemin_repertoire, f), sep=';')
    return df

# Chemin du répertoire contenant les fichiers CSV
chemin_repertoire = r'C:\Users\sofia\Desktop\4iabd\pa\data\gazs'

# Créer une liste de tous les fichiers CSV dans le répertoire
fichiers_csv = [f for f in os.listdir(chemin_repertoire) if f.endswith(".csv") and "2021" in f]

# Initialiser un pool de processus pour la parallélisation
pool = Pool(cpu_count())

# Lire chaque fichier CSV en parallèle et fusionner les DataFrames
dfs = pool.map(lire_csv, fichiers_csv)
df_polution_2021 = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame final
print(df_polution_2021)

